### Day 7: No Space Left On Device
You can hear birds chirping and raindrops hitting leaves as the expedition proceeds. Occasionally, you can even hear much louder sounds in the distance; how big do the animals get out here, anyway?

The device the Elves gave you has problems with more than just its communication system. You try to run a system update:

```
$ system-update --please --pretty-please-with-sugar-on-top

Error: No space left on device

```

Perhaps you can delete some files to make space for the update?

You browse around the filesystem to assess the situation and save the resulting terminal output (your puzzle input). For example:

```
$ cd /
$ ls
dir a
14848514 b.txt
8504156 c.dat
dir d
$ cd a
$ ls
dir e
29116 f
2557 g
62596 h.lst
$ cd e
$ ls
584 i
$ cd ..
$ cd ..
$ cd d
$ ls
4060174 j
8033020 d.log
5626152 d.ext
7214296 k
```

The filesystem consists of a tree of files (plain data) and directories (which can contain other directories or files). The outermost directory is called `/`. You can navigate around the filesystem, moving into or out of directories and listing the contents of the directory you're currently in.

Within the terminal output, lines that begin with `$` are commands you executed, very much like some modern computers:

- `cd` means change directory. This changes which directory is the current directory, but the specific result depends on the argument:
- `cd x` moves in one level: it looks in the current directory for the directory named `x` and makes it the current directory.
- `cd ..` moves out one level: it finds the directory that contains the current directory, then makes that directory the current directory.
- `cd /` switches the current directory to the outermost directory, `/`.
- `ls` means list. It prints out all of the files and directories immediately contained by the current directory:
- `123 abc` means that the current directory contains a file named `abc` with size `123`.
- `dir xyz` means that the current directory contains a directory named `xyz`.

Given the commands and output in the example above, you can determine that the filesystem looks visually like this:

```
- / (dir)
  - a (dir)
    - e (dir)
      - i (file, size=584)
    - f (file, size=29116)
    - g (file, size=2557)
    - h.lst (file, size=62596)
  - b.txt (file, size=14848514)
  - c.dat (file, size=8504156)
  - d (dir)
    - j (file, size=4060174)
    - d.log (file, size=8033020)
    - d.ext (file, size=5626152)
    - k (file, size=7214296)
```

Here, there are four directories: `/` (the outermost directory), `a` and `d` (which are in `/`), and `e` (which is in `a`). These directories also contain files of various sizes.

Since the disk is full, your first step should probably be to find directories that are good candidates for deletion. To do this, you need to determine the total size of each directory. The total size of a directory is the sum of the sizes of the files it contains, directly or indirectly. (Directories themselves do not count as having any intrinsic size.)

The total sizes of the directories above can be found as follows:

- The total size of directory `e` is `584` because it contains a single file `i` of size `584` and no other directories.
- The directory a has total size `94853` because it contains files `f` (size `29116`), `g` (size `2557`), and `h.lst` (size `62596`), plus file `i` indirectly (`a` contains `e` which contains `i`).
- Directory `d` has total size `24933642`.
- As the outermost directory, `/` contains every file. Its total size is `48381165`, the sum of the size of every file.

To begin, find all of the directories with a total size of at most `100000`, then calculate the sum of their total sizes. In the example above, these directories are `a` and `e`; the sum of their total sizes is `95437` (`94853 + 584`). (As in this example, this process can count files more than once!)

Find all of the directories with a total size of at most `100000`. What is the sum of the total sizes of those directories?



In [1]:
class Node:
    children = None
    name = None
    value = None

    def __init__(self, name):
        self.name = name

    def add_child(self, child):
        if self.children is not None:
            self.children.append(child)
        else:
            self.children = []
            self.children.append(child)

    def __str__(self):
        childstr = ""
        if self.children is not None and len(self.children) > 0:
            childstr = list(map(Node.__str__, self.children))
        return f"- {self.name}:{self.value} - {childstr}"

    def find_node_in_tree(self, next_dir):
        if self.name == next_dir:
            return self
        elif self.children is not None and len(self.children) > 0:
            for child in self.children:
                subnode = child.find_node_in_tree(next_dir)
                if subnode is not None:
                    return subnode
            return None
        else:
            return None

    def find_parent_of(self, search_child):
        if self.children is not None and len(self.children) > 0 and search_child in [selfchild.name for selfchild in
                                                                                     self.children]:
            return self
        elif self.children is not None and len(self.children) > 0:
            for selfchild in self.children:
                match = selfchild.find_parent_of(search_child)
                if match is not None:
                    return match
            return None
        else:
            return None


class Tree:
    root = Node("/")


def compute_dir_size(node):
    if node.children:
        return sum(map(compute_dir_size, node.children))
    elif node.value is not None:
        return int(node.value)
    else:
        return 0


with open("day7.txt") as file_content:
    lines = [line.strip() for line in file_content]
    tree = Tree()
    curdirname = "/"
    curdir = tree.root
    ariane="/"
    curcmd = None
    for line in lines:
        if line.startswith("$ cd"):
            curcmd = "cd"
            dir_to_move_to = line.replace("$ cd", "").strip()
            if dir_to_move_to == "..":
                ariane
                curdir = tree.root.find_parent_of(curdirname)
                print(f"moving from {curdirname} to parent {curdir.name}")
                curdirname = curdir.name
            elif curdirname != dir_to_move_to:
                curdir = tree.root.find_node_in_tree(dir_to_move_to)
                curdirname = dir_to_move_to

        elif line.startswith("$ ls"):
            curcmd = "ls"
        elif curcmd == "ls":
            # Display childs of curdir
            if line.startswith("dir"):
                childnode = Node(line.replace("dir", "").strip())
                curdir.add_child(childnode)
            else:
                childnode = Node(line.split(" ")[1])
                childnode.value = int(line.split(" ")[0])
                curdir.add_child(childnode)
    print(compute_dir_size(tree.root))


moving from lfrctthp to parent /
moving from rwvdvvsf to parent srf
moving from wvhhr to parent srf
moving from srf to parent lfrctthp
moving from lfrctthp to parent /
moving from jgltcw to parent bjfqsb
moving from bjfqsb to parent dvlb
moving from qdgqdtn to parent lfrctthp
moving from lnjln to parent sgddcfdn
moving from sgddcfdn to parent lfrctthp
moving from lfrctthp to parent /
moving from fhlnmw to parent rddpmj
moving from rddpmj to parent fnwsmj
moving from fnwsmj to parent dvlb
moving from rbwq to parent wlhfvwl
moving from wlhfvwl to parent dvlb
moving from wrgqqts to parent dvlb
moving from pnsbrd to parent hww
moving from hww to parent zhj
moving from rjgbm to parent nqv
moving from cqtnvzn to parent tlhttrgs
moving from tlhttrgs to parent jssbn
moving from jssbn to parent /


AttributeError: 'NoneType' object has no attribute 'name'